In [1]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import shuffle
import pandas as pd
import xgboost as xgb
import numpy as np
import random, os, datetime, pickle
import scipy
import matplotlib.pyplot as plt
import numpy as np
import csv


# tube.csv에 수술 시점 weight, height 추가
df0 = pd.read_csv('tube.csv')
df0.drop(columns=['weight', 'height'], inplace=True)
df = pd.read_csv('demography_revised.csv')
df = df0.merge(df[['opid', 'weight', 'height']], how='left', on='opid', suffixes=('_o',''))

df.loc[df['weight'] <= 0, 'weight'] = None
df.loc[df['height'] <= 0, 'height'] = None
df['age'] = df['age'].astype(int)
df = df.loc[df['airway_tube_type'] == 'plain']
# [nan 'plain' 'RAE(oral)' 'reinforced' 'LMA' 'T-tube' 'CobraPLA', 'double lumen tube' 'RAE(nasal)' 'laser' 'combitube' 'univent']

# age, sex, airway tube size 값이 없는 경우는 제외
df.dropna(subset=['age', 'airway_tube_size'], inplace=True)
df['sex'] = (df['sex'] == 'M')

# 나이 계산 -> age_cal 열에 추가
df_b = pd.read_csv('birth_sex.csv')
df_b.rename(columns={'생년월일':'birth_date'}, inplace=True)
df_b['birth_date'] = df_b['birth_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

df_o = pd.read_csv('opdates.csv')
df_o['opdate'] = df_o['opdate'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

df1 = pd.merge(df_o, df_b, how='inner', on='hid')
df1['age_cal'] = (df1['opdate'] - df1['birth_date'])/pd.Timedelta(days=365.2425)

df = pd.merge(df, df1[['opid', 'age_cal', 'opdate', 'birth_date']], how='inner', on='opid')

# inclusion criteria : 소아 10세 미만
df = df.loc[df['age_cal'] < 10-0.01]
df = df.loc[df['age_cal'] > 0]


# cuffed 여부와 fixed depth 추가
df_t = pd.read_csv('tube_type.csv')
df_t['cuffed'] = (df_t['cuffed'] == 1)

df_f = pd.read_csv('tube_fixed.csv')

# merge 하면서 cuffed 데이터가 없는 경우는 제외
df = df.merge(df_f, how='left', on='opid')
df = df.merge(df_t[['opid', 'cuffed']], how='inner', on='opid')

# 중복되는 hid 경우 제외 (첫번째 수술기록만 가져오기)
df = df.merge(df_o[['opid','hid']], how='inner', on='opid')
df = df.loc[df[['hid', 'opid']].groupby('hid')['opid'].idxmin()]
print(f'중복되는 hid는 첫번째 수술 외 제외: {len(df)}')


perc = np.mean(df['cuffed'].values)
print(f'소아 10세 미만에서 cuffed ETT 사용 비율: {perc:.3f}')
print(f'소아 10세 미만 최종 opid수: {len(df)}')

중복되는 hid는 첫번째 수술 외 제외: 34042
소아 10세 미만에서 cuffed ETT 사용 비율: 0.287
소아 10세 미만 최종 opid수: 34042


In [3]:
SEED = 98
INPUT_VARS = ['age_cal','sex','weight','height', 'cuffed']
TARGET_VAR = 'airway_tube_size'

random.seed(SEED)
df = shuffle(df)
df.reset_index(drop=True, inplace=True)
y = df[[TARGET_VAR]].values.flatten().astype(float)
c = df['opid'].values.flatten().astype(int)
#y_old = df[[OLD_VAR]].values.flatten().astype(float)
x = df.loc[:, INPUT_VARS].values.astype(float)

# 저장하기
#np.savez(f'dataset/ETT_size.npz', x=x, y=y, y_old=y_old, c=c)


# training set의 뒤쪽 20%를 test set 으로 사용
nsamp = len(y)
ntest = int(nsamp * 0.2)
ntrain = nsamp - ntest
x_test = x[-ntest:, :]
y_test = y[-ntest:]
#y_test_old = y_old[-ntest:]
x_train = x[:ntrain, :]
y_train = y[:ntrain]

print(f'x_train: {(x_train).shape}, x_test: {x_test.shape}')

x_train: (27234, 5), x_test: (6808, 5)


In [ ]:
# age-based formula에 따른 ETT size
OLD_VAR = 'old_tube_size'
# df[OLD_VAR] = np.round((df['age'] / 4 + 4) * 2) / 2
df[OLD_VAR] = df['age'].apply(lambda x: np.round((x / 4 + 4) * 2) / 2 if x >= 2 else (3.5 if x < 1 else 4)) 
df[OLD_VAR] = df.apply(lambda x: x[OLD_VAR] - 0.5 if x['cuffed'] else x[OLD_VAR], axis=1)

y_old = df[[OLD_VAR]].values.flatten().astype(float)
y_test_old = y[-ntest:]

# Cuffed model

In [8]:
train_mask = (x_train[:,4] == 1)
test_mask = (x_test[:,4] == 1)

x_train_c = x_train[train_mask][:,0:4]
y_train_c = y_train[train_mask]
x_test_c = x_test[test_mask][:,0:4]
y_test_c = y_test[test_mask]

In [12]:
# age (일단위)
param_dict = {
                #'learning_rate': [ 0.05, 0.07, 0.1], #[0.01, 0.03, 0.05],
                'max_depth': [4, 5, 7],#[3,4,5],
                'n_estimators': [100, 200, 300],
                #'n_estimators': [100],#[25, 50, 75, 100],
                #'n_estimators': [50],
                'subsample': [0.5, 0.8], #[0.5, 0.8, 1],
                'colsample_bytree': [0.5, 0.8], #[0.8, 1],
                #'gamma': [0.9], #[0.3, 0.5, 0.7, 0.9],
                #'scale_pos_weight': [5, 10], #[1,10,30,100]
            }
nfold = 5
gs = GridSearchCV(estimator=xgb.sklearn.XGBRegressor(),
                    n_jobs=-1,
                    verbose=3,
                    param_grid=param_dict, cv=nfold)
gs.fit(x_train_c, y_train_c)
model = gs.best_estimator_.get_booster()

print()
print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")

y_pred = gs.predict(x_test_c).flatten()
y_pred = np.round(y_pred * 2) / 2


print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test_c, y_pred):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test_c, y_pred):.3f}')
print(f'r2_score: {r2_score(y_test_c, y_pred):.3f}')
# accuracy
acc1 = np.mean(y_pred==y_test_c)
acc3 = np.mean((y_pred >= y_test_c-0.5) & (y_pred <= y_test_c+0.5))
print(f'acc: {acc1:.3f}')
print(f'acc(+-0.5mm): {acc3:.3f}')

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/home/painstudy/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 36 is smaller than n_iter=100. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



========= found hyperparameter =========
{'subsample': 0.8, 'n_estimators': 100, 'max_depth': 4, 'colsample_bytree': 0.5}
0.8978435598499278
--------------
new model
--------------
explained_variance_score: 0.895
mean_squared_errors: 0.120
r2_score: 0.895
acc: 0.676
acc(+-0.5mm): 0.956
[CV 1/5] END colsample_bytree=0.5, max_depth=5, n_estimators=100, subsample=0.8;, score=0.906 total time=   2.7s
[CV 2/5] END colsample_bytree=0.5, max_depth=7, n_estimators=200, subsample=0.5;, score=0.865 total time=   3.2s
[CV 5/5] END colsample_bytree=0.8, max_depth=5, n_estimators=100, subsample=0.8;, score=0.902 total time=   1.2s
[CV 5/5] END colsample_bytree=0.8, max_depth=7, n_estimators=100, subsample=0.5;, score=0.888 total time=   1.6s
[CV 1/5] END colsample_bytree=0.5, max_depth=4, n_estimators=100, subsample=0.5;, score=0.903 total time=   1.0s
[CV 1/5] END colsample_bytree=0.5, max_depth=5, n_estimators=200, subsample=0.5;, score=0.884 total time=   1.3s
[CV 1/5] END colsample_bytree=0.5,

# Uncuffed model

In [13]:
train_mask = (x_train[:,4] == 0)
test_mask = (x_test[:,4] == 0)

x_train_c = x_train[train_mask][:,0:4]
y_train_c = y_train[train_mask]
x_test_c = x_test[test_mask][:,0:4]
y_test_c = y_test[test_mask]

In [14]:
# age (일단위)
param_dict = {
                #'learning_rate': [ 0.05, 0.07, 0.1], #[0.01, 0.03, 0.05],
                'max_depth': [4, 5, 7],#[3,4,5],
                'n_estimators': [100, 200, 300],
                #'n_estimators': [100],#[25, 50, 75, 100],
                #'n_estimators': [50],
                'subsample': [0.5, 0.8], #[0.5, 0.8, 1],
                'colsample_bytree': [0.5, 0.8], #[0.8, 1],
                #'gamma': [0.9], #[0.3, 0.5, 0.7, 0.9],
                #'scale_pos_weight': [5, 10], #[1,10,30,100]
            }
nfold = 5
gs = GridSearchCV(estimator=xgb.sklearn.XGBRegressor(),
                    n_jobs=-1,
                    verbose=3,
                    param_grid=param_dict, cv=nfold)
gs.fit(x_train_c, y_train_c)
model = gs.best_estimator_.get_booster()

print()
print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")

y_pred = gs.predict(x_test_c).flatten()
y_pred = np.round(y_pred * 2) / 2


print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test_c, y_pred):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test_c, y_pred):.3f}')
print(f'r2_score: {r2_score(y_test_c, y_pred):.3f}')
# accuracy
acc1 = np.mean(y_pred==y_test_c)
acc3 = np.mean((y_pred >= y_test_c-0.5) & (y_pred <= y_test_c+0.5))
print(f'acc: {acc1:.3f}')
print(f'acc(+-0.5mm): {acc3:.3f}')

Fitting 5 folds for each of 36 candidates, totalling 180 fits

========= found hyperparameter =========
{'colsample_bytree': 0.5, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.8}
0.8703030600586213
--------------
new model
--------------
explained_variance_score: 0.835
mean_squared_errors: 0.155
r2_score: 0.835
acc: 0.555
acc(+-0.5mm): 0.967


# Cuffed + Uncuffed

In [16]:
# age (일단위)
param_dict = {
                #'learning_rate': [ 0.05, 0.07, 0.1], #[0.01, 0.03, 0.05],
                'max_depth': [3, 4, 5, 7],#[3,4,5],
                'n_estimators': [25, 50, 75, 100, 300],
                #'n_estimators': [100],#[25, 50, 75, 100],
                #'n_estimators': [50],
                'subsample': [0.5, 0.8, 1], #[0.5, 0.8, 1],
                'colsample_bytree': [0.5, 0.8, 1], #[0.8, 1],
                #'gamma': [0.9], #[0.3, 0.5, 0.7, 0.9],
                #'scale_pos_weight': [5, 10], #[1,10,30,100]
            }
nfold = 5
gs = GridSearchCV(estimator=xgb.sklearn.XGBRegressor(),
                n_jobs=-1,
                verbose=3,
                param_grid=param_dict, cv=nfold)
gs.fit(x_train, y_train)
model = gs.best_estimator_.get_booster()

print()
print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")

y_pred = gs.predict(x_test).flatten()
y_pred = np.round(y_pred * 2) / 2


print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
print(f'mean_squarSed_errors: {mean_squared_error(y_test, y_pred):.3f}')
print(f'r2_score: {r2_score(y_test, y_pred):.3f}')
# accuracy
acc1 = np.mean(y_pred==y_test)
acc3 = np.mean((y_pred >= y_test-0.5) & (y_pred <= y_test+0.5))
print(f'acc: {acc1:.3f}')
print(f'acc(+-0.5mm): {acc3:.3f}')

Fitting 5 folds for each of 180 candidates, totalling 900 fits

========= found hyperparameter =========
{'colsample_bytree': 1, 'max_depth': 4, 'n_estimators': 25, 'subsample': 0.8}
0.8828257876347063
--------------
new model
--------------
explained_variance_score: 0.861
mean_squared_errors: 0.140
r2_score: 0.861
acc: 0.595
acc(+-0.5mm): 0.966
[CV 2/5] END colsample_bytree=0.5, max_depth=5, n_estimators=100, subsample=0.8;, score=0.876 total time=   3.4s
[CV 4/5] END colsample_bytree=0.5, max_depth=7, n_estimators=100, subsample=0.5;, score=0.859 total time=   2.7s
[CV 5/5] END colsample_bytree=0.5, max_depth=7, n_estimators=300, subsample=0.8;, score=0.844 total time=   8.1s
[CV 4/5] END colsample_bytree=0.8, max_depth=7, n_estimators=100, subsample=0.8;, score=0.862 total time=   3.6s
[CV 4/5] END colsample_bytree=0.5, max_depth=4, n_estimators=100, subsample=0.5;, score=0.882 total time=   1.8s
[CV 2/5] END colsample_bytree=0.5, max_depth=5, n_estimators=200, subsample=0.5;, score